In [ ]:
def multiple_by_2(x:int) -> int:
    return x * 2 

In [ ]:
func = multiple_by_2

In [ ]:
func.__module__

In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from eda.eda_selection import _f_score, mrmr, MRMR_STRATEGY

In [ ]:
_f_score.__module__

In [ ]:
func.__name__

In [1]:
import polars as pl 
import json 

In [2]:
df = pl.DataFrame({
    "a":[{"test":1, "test2":2}, {"a":1, "df":2}]
})
df 

a
struct[2]
"{1,2}"
"{null,null}"


In [3]:
df.explode(columns=["a"])

InvalidOperationError: `explode` operation not supported for dtype `struct[2]`